In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp

In [4]:
roi_df = pd.read_csv(r'ROI_data.csv')
roi_df.set_index('Platform',inplace=True)

roi_mo_df = pd.read_csv(r'roi_mat.csv')
roi_mo_df.set_index('Unnamed: 0',inplace=True)
roi_mo_df = roi_mo_df.apply(lambda x: x/100)\

total_budget_original = 10000000

In [3]:
def optimize_allocation(df,estimate,budget,constraint_3=True):
    """
    

    Parameters
    ----------
    df : Dataframe
        ROI dataframe to be used
    estimate : integer
        ROI estimate to be used
    budget : integer
        Total company budget for marketing
    constraint_3 : boolean
        Keep or Remove 3rd constraint, keeps 3rd constraint by default

    Returns
    -------
    list
        List comprising of maximum returns and allocations for different marketing mediums

    """
    obj = np.array(df.iloc[estimate,:])
    A = np.zeros((3,df.shape[1]))
    A[0,:] = 1
    for column in df.columns:
        if column == 'Print':
            A[1,df.columns.get_loc(column)] = 1
        if column == 'TV':
            A[1,df.columns.get_loc(column)] = 1
        if column == 'SEO':
            A[2,df.columns.get_loc(column)] = -2
        if column == 'AdWords':
            A[2,df.columns.get_loc(column)] = -2
        if column == 'Facebook':
            A[1,df.columns.get_loc(column)] = -1
            A[2,df.columns.get_loc(column)] = 1
        if column == 'LinkedIn':
            A[2,df.columns.get_loc(column)] = 1
        if column == 'Instagram':
            A[2,df.columns.get_loc(column)] = 1
        if column == 'Snapchat':
            A[2,df.columns.get_loc(column)] = 1
        if column == 'Twitter':
            A[2,df.columns.get_loc(column)] = 1
        if column == 'Email':
            A[1,df.columns.get_loc(column)] = -1
    b = np.array([budget,0,0])
    sense = np.array(['<','<','>'])

    budgetModel = gp.Model()
    ub_constraint = np.ones(10)*3000000
    if constraint_3 == False:
        ub_constraint = np.ones(10)*budget
    budgetModX = budgetModel.addMVar(df.shape[1], lb=np.zeros(10), ub=ub_constraint)
    budgetModCon = budgetModel.addMConstr(A, budgetModX, sense, b)
    budgetModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

    budgetModel.Params.OutputFlag = 0 # tell gurobi to shut up!!

    budgetModel.optimize() # solve the LP
    
    constraints_sensitivity = np.zeros((3,3))
    constraints_sensitivity[:,0] = [con.Pi for con in budgetModCon]
    constraints_sensitivity[:,1] = [con.SARHSLow for con in budgetModCon]
    constraints_sensitivity[:,2] = [con.SARHSUp for con in budgetModCon]
    obj_sensitivity = np.zeros((df.shape[1],3))
    obj_sensitivity[:,0] = budgetModX.SAObjLow
    obj_sensitivity[:,2] = budgetModX.SAObjUp
    obj_sensitivity[:,1] = obj
    obj_sensitivity = obj_sensitivity.T

    return [budgetModel.objVal, budgetModX.x, constraints_sensitivity, obj_sensitivity]

In [7]:
max_returns, allocations, constraints, obj_sensitivity = optimize_allocation(roi_df,0,total_budget_original)
max_returns_2, allocations_2, constraints_2, obj_sensitivity_2 = optimize_allocation(roi_df,1,total_budget_original)


<ipython-input-3-715e68bbbe55>:65: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  constraints_sensitivity[:,0] = [con.Pi for con in budgetModCon]
<ipython-input-3-715e68bbbe55>:66: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  constraints_sensitivity[:,1] = [con.SARHSLow for con in budgetModCon]
<ipython-input-3-715e68bbbe55>:67: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.ar

array([      0., 3000000.,       0., 1000000.,       0.,       0.,
       3000000.,       0.,       0., 3000000.])

In [25]:
allocations_1_df = roi_df.append(pd.DataFrame([allocations],columns=roi_df.columns)).iloc[2,:]
allocations_1_df
allocations_2_df = roi_df.append(pd.DataFrame([allocations_2],columns=roi_df.columns)).iloc[2,:]
allocations_2_df
max_returns_2

456000.0

In [33]:
returns_1_2 = np.matmul(allocations_2.T,np.array(roi_df.iloc[0,:]))
returns_loss_1_2 = max_returns - returns_1_2
returns_2_1 = np.matmul(allocations.T,np.array(roi_df.iloc[1,:]))
returns_loss_2_1 = max_returns_2 - returns_2_1
max_returns_no3constraint, allocations_no3constraint, constraints_no3constraint, obj_sensitivity_no3constraint = optimize_allocation(roi_df,0,total_budget_original,constraint_3=False)
max_returns_2_no3constraint, allocations_2_no3constraint, constraints_2_no3constraint, obj_sensitivity_2_no3constraint = optimize_allocation(roi_df,1,total_budget_original,constraint_3=False)
returns_1_2_no3constraint = np.matmul(allocations_2_no3constraint.T,np.array(roi_df.iloc[0,:]))
returns_loss_1_2_no3constraint = max_returns_no3constraint - returns_1_2_no3constraint
returns_2_1_no3constraint = np.matmul(allocations_no3constraint.T,np.array(roi_df.iloc[1,:]))
returns_loss_2_1_no3constraint = max_returns_2_no3constraint - returns_2_1_no3constraint

df_cost = pd.DataFrame(np.zeros((2,3)),columns=['Constraint 3','Case 1','Case 2'])
df_cost

<ipython-input-3-715e68bbbe55>:65: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  constraints_sensitivity[:,0] = [con.Pi for con in budgetModCon]
<ipython-input-3-715e68bbbe55>:66: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  constraints_sensitivity[:,1] = [con.SARHSLow for con in budgetModCon]
<ipython-input-3-715e68bbbe55>:67: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.ar

,Constraint 3,Case 1,Case 1
0,0.0,0.0,0.0
1,0.0,0.0,0.0
